# VacationPy
----

Written in the Python 3.7.9 Environment

By Nicole Lund 

In partnership with Anne Niemiec.

The starter code and pseudo code for this challenge was provided and can be reviewed in python-api-challenge\assignment_source_files

I chose to use the cities.csv file provided in python-api-challenge\assignment_source_files for my source data.

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os

# Import API key
from api_keys import g_key
if g_key == "YOUR KEY HERE!":
    import sys
    sys.path.append(r"C:\Users\nlund\Documents\GitHub\untracked_files")
    from untracked_api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
input_file = r"input_data/cities.csv"
df_wx = pd.read_csv(input_file)
df_wx.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


In [15]:
# Remove invalid humidity data
df_wx = df_wx.loc[df_wx["Humidity"] <= 100, ["City","Lat","Lng","Max Temp","Humidity","Cloudiness","Wind Speed","Country","Date"]]
df_wx.describe()

,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
count,547.000000,547.000000,547.000000,547.000000,547.000000,547.000000,5.470000e+02
mean,19.710402,20.861609,64.975119,67.265082,49.495430,8.657057,1.558379e+09
std,33.637627,90.285396,18.721163,21.723477,36.887333,5.859917,7.826709e+01
min,-54.810000,-173.230000,4.920000,4.000000,0.000000,0.540000,1.558378e+09
25%,-6.315000,-60.820000,53.940000,52.000000,14.000000,4.680000,1.558379e+09
50%,21.430000,21.550000,64.990000,71.000000,47.000000,7.000000,1.558379e+09
75%,47.895000,103.165000,80.600000,86.000000,81.000000,11.410000,1.558379e+09
max,78.220000,179.320000,104.100000,100.000000,100.000000,46.080000,1.558379e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [16]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Customize the size of the figure
figure_layout = {
    'width': '600px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

In [17]:
# Collect data of interest from DataFrame
df_coordinates = df_wx[['Lat','Lng']]
s_humidity = df_wx.Humidity

In [18]:
# Create a Humidity Heatmap
heat_layer = gmaps.heatmap_layer(df_coordinates,
    weights=s_humidity, 
    dissipating=False, 
    max_intensity=s_humidity.max(),
    point_radius = 2)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [20]:
# Collect wx data for cities with ideal conditions
df_ideal_wx = df_wx.loc[
    (df_wx["Max Temp"] < 80) &
    (df_wx["Max Temp"] > 70) &
    (df_wx["Wind Speed"] < 10) &
    (df_wx["Cloudiness"] < 15),
    ["City","Cloudiness",
     "Country","Date","Humidity",
    "Lat","Lng","Max Temp","Wind Speed"]
]

In [21]:
# Remove rows with NaN values
df_ideal_wx.dropna(inplace=True)
df_ideal_wx.describe()

,Cloudiness,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
count,12.000000,1.200000e+01,12.000000,12.000000,12.000000,12.000000,12.000000
mean,2.166667,1.558379e+09,69.166667,2.881667,53.965833,75.506667,4.597500
std,3.785939,1.025758e+02,22.670900,22.053344,61.303681,2.908199,2.790976
min,0.000000,1.558378e+09,26.000000,-30.170000,-50.220000,70.980000,1.070000
25%,0.000000,1.558379e+09,55.500000,-15.777500,32.477500,73.320000,2.445000
50%,0.000000,1.558379e+09,69.000000,7.925000,54.695000,75.880000,4.765000
75%,2.500000,1.558379e+09,89.000000,17.235000,103.050000,77.475000,5.857500
max,12.000000,1.558379e+09,99.000000,35.170000,126.060000,79.080000,9.170000


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [22]:
# Initialize Google Places Search
hotel_df = df_ideal_wx.copy()

params = {
    "keyword": "Hotel",
    "radius": 5000,
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

In [23]:
# Collect hotel names
hotel_names = []

for index, row in hotel_df.iterrows():
    # Define search coordinates and add to params
    coord = row[["Lat","Lng"]].tolist()
    params["location"] = f"{coord[0]}, {coord[1]}"
    
    # run a request using params dictionary
    response = requests.get(base_url, params=params).json()
    # print(json.dumps(response, indent=4,sort_keys=True))
    try:
        hotel_names.append(response["results"][0]["name"])
    except IndexError:
        hotel_names.append("No Hotels Found")
        print(f'{row["City"]}, skipped due to no hotels')

vaini, skipped due to no hotels
dingle, skipped due to no hotels
birjand, skipped due to no hotels
baylo, skipped due to no hotels


In [24]:
# Add hotel name column to the DataFrame
hotel_df["Hotel Name"] = hotel_names
hotel_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32,Camaleao Hostel
87,ambilobe,2,MG,1558378770,92,-13.19,49.05,74.76,4.52,Hotel National
88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50,Hotel Castelo
138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06,Palissandre Cote Ouest resort & SPA
139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59,No Hotels Found


In [25]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [26]:
# Add marker layer ontop of heat map
marker_locations = hotel_df[["Lat","Lng"]]

# Display figure
# Create a marker_layer using the poverty list to fill the info box
markers = gmaps.marker_layer(marker_locations,
    info_box_content=hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…